In [2]:
import pandas as pd
from bs4 import BeautifulSoup as soup
import requests

rawpath = '../raw'

In [3]:
df = pd.read_excel(f"{rawpath}/Austest.xlsx",names=['name'], dtype='string')
df = df.apply(lambda x: x.str.strip())
df

,name
0,3L-Verlag


In [4]:
# Basisurl für GND Dateien und dictionary für die Bestandteile der URL 
base_url = 'https://services.dnb.de/sru/authorities?'
params = {'recordSchema' : 'MARC21-xml',
          'operation': 'searchRetrieve',
          'version': '1.1',
          'maximumRecords': '100',
         }

In [5]:
def gnd_abfrage(row):
    params.update({'query': f'KOE="{row.name}" AND BBG=Tb*'})
    response = requests.get(base_url, params=params)
    response_xml = soup(response.content, features="xml")

    try:
        matches = list()
        for record in response_xml.find_all('record', {'type':'Authority'}):
            record_match = dict()
            #idn bestimmen
            try:
                record_match['idn'] = record.find('controlfield', {'tag': '001'}).string.strip()
            except:
                record_match['idn'] = None


            matches.append(record_match)
        return matches
    except Exception as e:
        return "fehler %r" % e

In [6]:
df['idn'] = df.apply(gnd_abfrage, axis=1)
df

,name,idn
0,3L-Verlag,"[{'idn': '1237247934'}, {'idn': '123696005X'},..."


In [6]:
df.to_excel('aussteller1.xlsx')
#test